# Simulation with RES and Python
## Pre-Requisite
   * Python 3
   * Support of PySpark

In [4]:
import os
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession

cwd = os.getcwd()
print(cwd)
# Download ODM Library
#!curl -o {cwd}/j2ee_connector-1_5-fr.jar http://159.122.179.123:31329/download/lib/ODM8920/j2ee_connector-1_5-fr.jar
#!curl -o {cwd}/jrules-engine.jar http://159.122.179.123:31329/download/lib/ODM8920/jrules-engine.jar
#!curl -o {cwd}/jrules-res-execution.jar http://159.122.179.123:31329/download/lib/ODM8920/jrules-res-execution-memory.jar

os.environ['PYSPARK_SUBMIT_ARGS'] = "--jars local:"+cwd+"/cdira1533889035609_81c1f938_f0ba_4f56_95a4_2079d08061b7.jar,local:"+cwd+"/loanvalidation-xom.jar,local:"+cwd+"/jrules-engine.jar,local:"+cwd+"/j2ee_connector-1_5-fr.jar,local:"+cwd+"/jrules-res-execution.jar pyspark-shell"



sc = SparkSession.builder.getOrCreate()


/home/jovyan/work/simulationloan


In [132]:
import json
import pandas as pd
import numpy as np
from pyspark.sql import SQLContext
# Create a SParkSQL Context to load the data in a dataframe
sql = SQLContext(sc.sparkContext)
new_decisions_pd = pd.read_csv(
    "loanvalidation-input.csv",
    decimal=",",
    header = 0,
    skiprows=[0,1,2,3,5],
    sep=';',
    converters={'Scenario ID':str,'zip code':str},
    dtype={'number of monthly payments': int})
request_df = sql.createDataFrame(new_decisions_pd)
request_df.printSchema()
request_df.show(10)
firstrow = request_df.first

root
 |-- Scenario ID: string (nullable = true)
 |-- description: double (nullable = true)
 |-- first name: string (nullable = true)
 |-- last name: string (nullable = true)
 |-- birth date: string (nullable = true)
 |-- SSNCode: string (nullable = true)
 |-- credit score: long (nullable = true)
 |-- yearly income: long (nullable = true)
 |-- zip code: string (nullable = true)
 |-- start date: string (nullable = true)
 |-- number of monthly payments: long (nullable = true)
 |-- amount: long (nullable = true)
 |-- Loan to Value: double (nullable = true)

+-----------+-----------+----------+---------+----------+-----------+------------+-------------+--------+----------+--------------------------+------+-------------+
|Scenario ID|description|first name|last name|birth date|    SSNCode|credit score|yearly income|zip code|start date|number of monthly payments|amount|Loan to Value|
+-----------+-----------+----------+---------+----------+-----------+------------+-------------+--------+-----

In [235]:
flowTask=None
userData=None
def CreateODMSession(sc):
    if not hasattr(CreateODMSession, "fac"):
        factoryConfig = sc._jvm.ilog.rules.res.session.IlrJ2SESessionFactory.createDefaultConfig()
        xuConfig = factoryConfig.getXUConfig();
        xuConfig.setLogAutoFlushEnabled(True);
        xuConfig.getPersistenceConfig().setPersistenceType(sc._jvm.ilog.rules.res.session.config.IlrPersistenceType.MEMORY);
        xuConfig.getManagedXOMPersistenceConfig().setPersistenceType(sc._jvm.ilog.rules.res.session.config.IlrPersistenceType.MEMORY);
        CreateODMSession.fac=sc._jvm.ilog.rules.res.session.IlrJ2SESessionFactory(factoryConfig)    
    return CreateODMSession.fac


def computeDomainCache(ruleSession):
    domainValuesMap = None
    # Invoke the function (that is embedded in the ruleset) to build the domain values cache.
    invokeFunctionResult = ruleSession.getXURuleEngineSession().invokeFunction("com_ibm_rules_cdi_computeDomainsFunc", None);
    #Check function result.
    if (invokeFunctionResult != None):
        domainValuesMap =  invokeFunctionResult
    return domainValuesMap;

def computeMetrics(ruleSession, domainsValuesCache):
    xomParameters = None
    metricMap = None
    direction = sc._jvm.ilog.rules.res.model.IlrRulesetParameter.DIRECTION_IN + sc._jvm.ilog.rules.res.model.IlrRulesetParameter.DIRECTION_INOUT + sc._jvm.ilog.rules.res.model.IlrRulesetParameter.DIRECTION_OUT
    xomParameters = ruleSession.getXURuleEngineSession().getXOMParameters(direction)
    if (xomParameters != None and not xomParameters.isEmpty()):
        # Prepare function parameters.
        functionparams= sc.sparkContext._gateway.new_array(sc._jvm.java.lang.Object, len(xomParameters)+1)
        aList = sc._jvm.java.util.ArrayList()
        aList.append(domainsValuesCache)
         # Sort the ruleset parameters by name.
        sortedKeys = sc._jvm.java.util.TreeSet(xomParameters.keySet());
        for key in sortedKeys:
            aList.append(xomParameters[key])

        functionparams = aList.toArray(functionparams)
   

    # Invoke the function (that is supposed to be embedded in the ruleset) to compute the metrics.
    invokeFunctionResult = ruleSession.getXURuleEngineSession().invokeFunction("com_ibm_rules_cdi_computeMetricsFunc", functionparams);

    # Check function result.
    if (invokeFunctionResult != None):
        metricMap = invokeFunctionResult
#    print("Metrics : ",metricMap)
    return metricMap

def executeRules(factory,rsPath,inputParameters):
    rsPath = sc._jvm.ilog.rules.res.model.IlrPath.parsePath(rsPath)
    sessionRequest = factory.createRequest()
    sessionRequest.setRulesetPath(rsPath)
    if (flowTask != None):
        sessionRequest.setTaskName(flowTask)
    if (userData != None):
        sessionRequest.setUserData(userData)
    sessionRequest.setTraceEnabled(True)
    sessionRequest.getTraceFilter().setInfoAllFilters(False)
    sessionRequest.setInputParameters(inputParameters)


    # Execute rules
    # Create a session request object
    # TODO Need to manage 
    #    the Try / Catch session.close
    #    The Taskname session.execute(inputParameters, TASKNAME)
    session = factory.createStatefulSession(rsPath, None, None, True)
    sessionResponse = session.execute(sessionRequest.getInputParameters(), None)
    domainCache = computeDomainCache(session)
    metrics = computeMetrics(session,domainCache)
    outParameters=sessionResponse.getOutputParameters()
    session.close()
    return outParameters,metrics

def printResults(metrics,outParameters):
    print("Decision Executions return this parameters : ")
    print(metrics)
    for key in outParameters:
        print (key, "corresponds to", outParameters[key])
    print("Nb of Rules fire : ",outParameters['ilog.rules.firedRulesCount'])
    print("--------")
factory=CreateODMSession(sc)


for row in request_df.rdd.collect():
    sFormat =  sc._jvm.java.text.SimpleDateFormat("MM/dd/YYYY");  
    inputParameters = sc._jvm.java.util.HashMap();
    borrower =  sc._jvm.loan.Borrower(row['first name'],
                                      row['last name'], 
                                      sFormat.parse(row['birth date']),
                                      row['SSNCode'])
    borrower.setCreditScore(row['credit score'])
    borrower.setZipCode(row['zip code'])
    borrower.setYearlyIncome(row['yearly income'])

    loan = sc._jvm.loan.LoanRequest(sFormat.parse(row['start date']),row['number of monthly payments'], row['amount'],row['Loan to Value'])
    inputParameters["loan"]=loan
    inputParameters["borrower"]=borrower
    
    # Init ODM Session 
    outParameters,metrics=executeRules(factory,"/cdira1533889035609_81c1f938_f0ba_4f56_95a4_2079d08061b7/1.0/cdirs1533889035609_81c1f938_f0ba_4f56_95a4_2079d08061b7/1.0",inputParameters)
#    printResults(metrics,outParameters)


